### 🛠️ Workflow (Feature Engineering & Data Transformation)

* **Load** merged dataset (`train_final`, `test_final`)
* **Feature Creation**:

  * `add_time_features(df)` — Extract year, month, day, day of week
  * `add_lag_features(df, lags=[1, 7, 14, 28])` — Incorporate temporal dynamics
  * `add_rolling_features(df, windows=[7, 14, 28])` — Smooth patterns and trend capture
  * `add_external_features(df)` — Holidays, promotions, oil prices
* **Encoding & Transformation**:

  * One‑hot encode categorical variables (`state`, `family`, `type`, etc.)
  * Impute missing values (`transactions` column)
  * Scale and normalize numeric fields
* **Data Split & Save**:

  * Split into training & validation sets
  * Persist transformers and pipelines for deployment


In [1]:
import os

In [2]:
%pwd

'd:\\MyCase\\Projects\\DSAI\\portfolio\\Sales_Forecasting_and_Analytics\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\MyCase\\Projects\\DSAI\\portfolio\\Sales_Forecasting_and_Analytics'

<p> Loading the Data </p>

In [5]:
import pandas as pd

try:
  train_df = pd.read_csv("artifacts/data_preprocessing/train_merged.csv")
  test_df = pd.read_csv("artifacts/data_preprocessing/test_merged.csv")

except FileNotFoundError:
  print("Files not found. Please ensure the data files are in the correct directory.")

C:\Users\rosha\AppData\Local\Temp\ipykernel_34332\2661553361.py:4: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv("artifacts/data_preprocessing/train_merged.csv")


In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class FeatureEngineeringAndDataTransformationConfig:
    root_dir: Path
    input_train_file: str
    input_test_file: str
    train_file: str
    test_file: str
    scaler_file: str


In [7]:
from ml_service.constants import *
from ml_service.utils.main_utils import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(self, config_filepath: str):
        """Initialize the configuration manager.

        Args:
            config_filepath (str): Path to the main configuration file (YAML).
        """
        self.config = read_yaml(config_filepath)
        create_directories([self.config.artifacts_root])

    def get_feature_engineering_and_data_transformation_config(self) -> FeatureEngineeringAndDataTransformationConfig:
        """Get the configuration for feature engineering and data transformation."""
        config = self.config.features_dataTransformation
        feature_config = FeatureEngineeringAndDataTransformationConfig(
            root_dir=Path(config.root_dir),
            input_train_file=config.input_train_file,
            input_test_file=config.input_test_file,
            train_file=config.train_final,
            test_file=config.test_final,
            scaler_file=config.scaler_file
        )
        create_directories([feature_config.root_dir])
        return feature_config


### Feature Engineering

In [9]:
train_df.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,transactions,dcoilwtico,type_y
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,NaN,NaN,Holiday


In [10]:
test_df.head()

,id,date,store_nbr,family,onpromotion,city,state,type_x,cluster,transactions,dcoilwtico,type_y
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,NaN,46.8,NaN
1,3000889,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13,NaN,46.8,NaN
2,3000890,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13,NaN,46.8,NaN
3,3000891,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13,NaN,46.8,NaN
4,3000892,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13,NaN,46.8,NaN


In [11]:
train_df.isnull().sum()

id                    0
date                  0
store_nbr             0
family                0
sales                 0
onpromotion           0
city                  0
state                 0
type_x                0
cluster               0
transactions     246543
dcoilwtico         1782
type_y          2680128
dtype: int64

<p>Before extracting time features, we have a large number of missing values in <code>type_y</code>, so we will replace them with <strong>"Regular Day"</strong> to maintain consistency in the data.</p>

In [12]:
train_df["type_y"] = train_df["type_y"].fillna("Regular Day")
test_df["type_y"] = test_df["type_y"].fillna("Regular Day")

In [13]:
train_df['type_y'].value_counts()

type_y
Regular Day    2680128
Holiday         206712
Event            62370
Additional       44550
Transfer         14256
Bridge            5346
Name: count, dtype: int64

In [14]:
train_df.isnull().sum()

id                   0
date                 0
store_nbr            0
family               0
sales                0
onpromotion          0
city                 0
state                0
type_x               0
cluster              0
transactions    246543
dcoilwtico        1782
type_y               0
dtype: int64

In [15]:
train_df["dcoilwtico"]

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
           ...  
3013357    47.57
3013358    47.57
3013359    47.57
3013360    47.57
3013361    47.57
Name: dcoilwtico, Length: 3013362, dtype: float64

<p>I have Used <code>bfill()</code> to fill missing <code>dcoilwtico</code> values, ensuring continuity in the data.</p>

In [16]:
train_df["dcoilwtico"] = train_df["dcoilwtico"].bfill()

In [17]:
train_df["dcoilwtico"].isna().sum()

np.int64(0)

In [18]:
test_df['type_y'].unique()

array(['Regular Day', 'Holiday'], dtype=object)

In [19]:
train_df['type_y'].unique()

array(['Holiday', 'Regular Day', 'Additional', 'Transfer', 'Event',
       'Bridge'], dtype=object)

1. Time Features

In [20]:
def add_time_features(df):
    df["date"] = pd.to_datetime(df["date"]) 
    df["year"] = df["date"].dt.year
    df["month"] = df["date"].dt.month
    df["day"] = df["date"].dt.day
    df["day_of_week"] = df["date"].dt.dayofweek
    df["is_weekend"] = (df["day_of_week"] >= 5).astype(int)
    df["day_of_year"] = df["date"].dt.dayofyear
    df["is_month_start"] = df["date"].dt.is_month_start.astype(int)
    df["is_month_end"] = df["date"].dt.is_month_end.astype(int)
    return df

train_df = add_time_features(train_df)
test_df = add_time_features(test_df)



In [21]:
train_df.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,...,dcoilwtico,type_y,year,month,day,day_of_week,is_weekend,day_of_year,is_month_start,is_month_end
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,...,93.14,Holiday,2013,1,1,1,0,1,1,0
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,...,93.14,Holiday,2013,1,1,1,0,1,1,0
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,...,93.14,Holiday,2013,1,1,1,0,1,1,0
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,...,93.14,Holiday,2013,1,1,1,0,1,1,0
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,...,93.14,Holiday,2013,1,1,1,0,1,1,0


2. Interaction Features

In [22]:
train_df["onpromotion_trend"] = train_df["onpromotion"] * train_df["day_of_year"]
train_df["month_sales_interaction"] = train_df["month"] * train_df["sales"]

train_df.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,...,year,month,day,day_of_week,is_weekend,day_of_year,is_month_start,is_month_end,onpromotion_trend,month_sales_interaction
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,...,2013,1,1,1,0,1,1,0,0,0.0
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,...,2013,1,1,1,0,1,1,0,0,0.0
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,...,2013,1,1,1,0,1,1,0,0,0.0
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,...,2013,1,1,1,0,1,1,0,0,0.0
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,...,2013,1,1,1,0,1,1,0,0,0.0


In [23]:
# Ensure date is datetime and data is sorted

train_df["date"] = pd.to_datetime(train_df["date"])
train_df = train_df.sort_values(["store_nbr", "family", "date"])

# Define a group object (one series per store-family)
group = train_df.groupby(["store_nbr", "family"])

# Create the rolling features
# 7‑day rolling std of sales
train_df["sales_roll_std_7"] = (
    group["sales"]
    .transform(lambda s: s.rolling(window=7, min_periods=1).std())
)

# 30‑day rolling mean and std
train_df["sales_roll_mean_30"] = (
    group["sales"]
    .transform(lambda s: s.rolling(window=30, min_periods=1).mean())
)
train_df["sales_roll_std_30"] = (
    group["sales"]
    .transform(lambda s: s.rolling(window=30, min_periods=1).std())
)

# 60‑day rolling mean and std
train_df["sales_roll_mean_60"] = (
    group["sales"]
    .transform(lambda s: s.rolling(window=60, min_periods=1).mean())
)
train_df["sales_roll_std_60"] = (
    group["sales"]
    .transform(lambda s: s.rolling(window=60, min_periods=1).std())
)

# Create the expanding features

train_df["sales_expanding_mean"] = (
    group["sales"].transform(lambda s: s.expanding(min_periods=1).mean())
)
train_df["sales_expanding_max"] = (
    group["sales"].transform(lambda s: s.expanding(min_periods=1).max())
)
train_df["sales_expanding_min"] = (
    group["sales"].transform(lambda s: s.expanding(min_periods=1).min())
)


3. Cyclical Encoding

Time features like **month** and **day_of_week** are cyclical (December connects to January, Sunday connects to Monday). 

**Why?**  
Makes the model understand the **circular nature of time**, yielding smoother and more accurate temporal patterns for forecasting.


In [24]:
import numpy as np

train_df["month_sin"] = np.sin(2 * np.pi * train_df["month"] / 12)
train_df["month_cos"] = np.cos(2 * np.pi * train_df["month"] / 12)

train_df["day_of_week_sin"] = np.sin(2 * np.pi * train_df["day_of_week"] / 7)
train_df["day_of_week_cos"] = np.cos(2 * np.pi * train_df["day_of_week"] / 7)

test_df["month_sin"] = np.sin(2 * np.pi * test_df["month"] / 12)
test_df["month_cos"] = np.cos(2 * np.pi * test_df["month"] / 12)

test_df["day_of_week_sin"] = np.sin(2 * np.pi * test_df["day_of_week"] / 7)
test_df["day_of_week_cos"] = np.cos(2 * np.pi * test_df["day_of_week"] / 7)

train_df.drop(["month", "day_of_week"], axis=1, inplace=True)
test_df.drop(["month", "day_of_week"], axis=1, inplace=True)

train_df.head()


,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,...,sales_roll_std_30,sales_roll_mean_60,sales_roll_std_60,sales_expanding_mean,sales_expanding_max,sales_expanding_min,month_sin,month_cos,day_of_week_sin,day_of_week_cos
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,...,NaN,0.000000,NaN,0.000000,0.0,0.0,0.5,0.866025,0.781831,0.623490
1782,1782,2013-01-02,1,AUTOMOTIVE,2.0,0,Quito,Pichincha,D,13,...,1.414214,1.000000,1.414214,1.000000,2.0,0.0,0.5,0.866025,0.974928,-0.222521
3564,3564,2013-01-03,1,AUTOMOTIVE,3.0,0,Quito,Pichincha,D,13,...,1.527525,1.666667,1.527525,1.666667,3.0,0.0,0.5,0.866025,0.433884,-0.900969
5346,5346,2013-01-04,1,AUTOMOTIVE,3.0,0,Quito,Pichincha,D,13,...,1.414214,2.000000,1.414214,2.000000,3.0,0.0,0.5,0.866025,-0.433884,-0.900969
7128,7128,2013-01-05,1,AUTOMOTIVE,5.0,0,Quito,Pichincha,D,13,...,1.816590,2.600000,1.816590,2.600000,5.0,0.0,0.5,0.866025,-0.974928,-0.222521


In [25]:
cat_columns = train_df.select_dtypes(include=['object', 'category']).columns.tolist()
print("Categorical columns:", cat_columns)

Categorical columns: ['family', 'city', 'state', 'type_x', 'type_y']


In [26]:
print(train_df['type_y'].unique().tolist())

['Holiday', 'Regular Day', 'Additional', 'Transfer', 'Event', 'Bridge']


In [27]:
print(test_df['type_y'].unique().tolist())

['Regular Day', 'Holiday']


### Encode Categorical Columns

<img src="https://i.imgur.com/n8GuiOO.png" width="640">

Let's one-hot encode categorical columns.

In [28]:
encoded = pd.get_dummies(
    train_df[cat_columns],
    drop_first=True,
    dtype=int
)
train_df = pd.concat([train_df, encoded], axis=1)
# Transforming the test dataset similarly
encoded = pd.get_dummies(
    test_df[cat_columns],
    drop_first=True,
    dtype=int
)
test_df = pd.concat([test_df, encoded], axis=1)

# encoded = pd.get_dummies(combined[cat_columns], drop_first=True, dtype=int)
# combined = pd.concat([combined, encoded], axis=1)


In [29]:
print(train_df.columns.to_list())

['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion', 'city', 'state', 'type_x', 'cluster', 'transactions', 'dcoilwtico', 'type_y', 'year', 'day', 'is_weekend', 'day_of_year', 'is_month_start', 'is_month_end', 'onpromotion_trend', 'month_sales_interaction', 'sales_roll_std_7', 'sales_roll_mean_30', 'sales_roll_std_30', 'sales_roll_mean_60', 'sales_roll_std_60', 'sales_expanding_mean', 'sales_expanding_max', 'sales_expanding_min', 'month_sin', 'month_cos', 'day_of_week_sin', 'day_of_week_cos', 'family_BABY CARE', 'family_BEAUTY', 'family_BEVERAGES', 'family_BOOKS', 'family_BREAD/BAKERY', 'family_CELEBRATION', 'family_CLEANING', 'family_DAIRY', 'family_DELI', 'family_EGGS', 'family_FROZEN FOODS', 'family_GROCERY I', 'family_GROCERY II', 'family_HARDWARE', 'family_HOME AND KITCHEN I', 'family_HOME AND KITCHEN II', 'family_HOME APPLIANCES', 'family_HOME CARE', 'family_LADIESWEAR', 'family_LAWN AND GARDEN', 'family_LINGERIE', 'family_LIQUOR,WINE,BEER', 'family_MAGAZINES', 'family_ME

In [30]:
train_df.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,...,state_Tungurahua,type_x_B,type_x_C,type_x_D,type_x_E,type_y_Bridge,type_y_Event,type_y_Holiday,type_y_Regular Day,type_y_Transfer
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,...,0,0,0,1,0,0,0,1,0,0
1782,1782,2013-01-02,1,AUTOMOTIVE,2.0,0,Quito,Pichincha,D,13,...,0,0,0,1,0,0,0,0,1,0
3564,3564,2013-01-03,1,AUTOMOTIVE,3.0,0,Quito,Pichincha,D,13,...,0,0,0,1,0,0,0,0,1,0
5346,5346,2013-01-04,1,AUTOMOTIVE,3.0,0,Quito,Pichincha,D,13,...,0,0,0,1,0,0,0,0,1,0
7128,7128,2013-01-05,1,AUTOMOTIVE,5.0,0,Quito,Pichincha,D,13,...,0,0,0,1,0,0,0,0,1,0


In [31]:
train_df['transactions'].describe()

count    2.766819e+06
mean     1.696450e+03
std      9.659076e+02
min      5.000000e+00
25%      1.046000e+03
50%      1.394000e+03
75%      2.081000e+03
max      8.359000e+03
Name: transactions, dtype: float64

In [32]:
train_df['transactions'].isna().sum()

np.int64(246543)

### 💡 Handling `transactions` Nulls

In retail data, `NaN` often means **no activity** rather than an error.so, we will fill with **0** unless we have evidence it's a recording error.

**Why?**
Keeps it simple, sensible, and avoids adding noise from guesses. We can revisit and test imputation later if needed.


In [33]:
train_df["transactions"] = train_df["transactions"].fillna(0)  
test_df["transactions"] = test_df["transactions"].fillna(0)

In [34]:
train_df.isnull().sum().sum()

np.int64(5346)

In [35]:
train_df["is_holiday"] = train_df["type_y_Holiday"]
# test_df["is_holiday"] = test_df["type_y_Holiday"]

In [36]:
train_df.describe()

,id,date,store_nbr,sales,onpromotion,cluster,transactions,dcoilwtico,year,day,...,type_x_B,type_x_C,type_x_D,type_x_E,type_y_Bridge,type_y_Event,type_y_Holiday,type_y_Regular Day,type_y_Transfer,is_holiday
count,3.013362e+06,3013362,3.013362e+06,3.013362e+06,3.013362e+06,3.013362e+06,3.013362e+06,3.013362e+06,3.013362e+06,3.013362e+06,...,3.013362e+06,3.013362e+06,3.013362e+06,3.013362e+06,3.013362e+06,3.013362e+06,3.013362e+06,3.013362e+06,3.013362e+06,3.013362e+06
mean,1.501547e+06,2015-04-24 23:20:49.674748160,2.750000e+01,3.583249e+02,2.605846e+00,8.481481e+00,1.557653e+03,6.784705e+01,2.014839e+03,1.565405e+01,...,1.481481e-01,2.777778e-01,3.333333e-01,7.407407e-02,1.774098e-03,2.069781e-02,6.859846e-02,8.894145e-01,4.730928e-03,6.859846e-02
min,0.000000e+00,2013-01-01 00:00:00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,2.619000e+01,2.013000e+03,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,7.533402e+05,2014-02-28 00:00:00,1.400000e+01,0.000000e+00,0.000000e+00,4.000000e+00,9.300000e+02,4.637000e+01,2.014000e+03,8.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
50%,1.501334e+06,2015-04-25 00:00:00,2.750000e+01,1.100000e+01,0.000000e+00,8.500000e+00,1.332000e+03,5.328568e+01,2.015000e+03,1.600000e+01,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
75%,2.249329e+06,2016-06-19 00:00:00,4.100000e+01,1.960000e+02,0.000000e+00,1.300000e+01,1.979000e+03,9.578000e+01,2.016000e+03,2.300000e+01,...,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
max,3.000887e+06,2017-08-15 00:00:00,5.400000e+01,1.247170e+05,7.410000e+02,1.700000e+01,8.359000e+03,1.106200e+02,2.017000e+03,3.100000e+01,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
std,8.653808e+05,NaN,1.558579e+01,1.105166e+03,1.222797e+01,4.649735e+00,1.035781e+03,2.567804e+01,1.344091e+00,8.788623e+00,...,3.552468e-01,4.479033e-01,4.714046e-01,2.618914e-01,4.208267e-02,1.423707e-01,2.527701e-01,3.136181e-01,6.861886e-02,2.527701e-01


In [37]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3013362 entries, 0 to 3013229
Columns: 111 entries, id to is_holiday
dtypes: datetime64[ns](1), float64(16), int32(3), int64(86), object(5)
memory usage: 2.5+ GB


In [38]:
train_df.shape, test_df.shape

((3013362, 111), (28512, 95))

### Scale Numeric Values

<p>Since all the values are in different ranges, we'll scale them to a 0–1 range so the model can better understand the data and learn more effectively and efficiently.</p>

In [39]:
from sklearn.preprocessing import MinMaxScaler

scale_columns = [
    "onpromotion",
    "transactions",
    "dcoilwtico",
    "sales_expanding_mean",
    "sales_expanding_max",
    "sales_expanding_min",
    "onpromotion_trend",
    "month_sales_interaction",
]

scaler = MinMaxScaler()


train_df[scale_columns] = scaler.fit_transform(train_df[scale_columns])

In [40]:
import joblib
import os

artifacts_dir = "artifacts/model"
os.makedirs(artifacts_dir, exist_ok=True)

scaler_path = os.path.join(artifacts_dir, "scaler.joblib")
joblib.dump(scaler, scaler_path)

print(f"✅ Scaler saved at: {scaler_path}")

✅ Scaler saved at: artifacts/model\scaler.joblib


In [ ]:
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.preprocessing import MinMaxScaler
from pathlib import Path


class FeatureEngineeringAndDataTransformation:
    def __init__(self, train_file, test_file, output_dir, scale_file, dtype_spec=None):
        self.train_file = train_file
        self.test_file = test_file
        self.output_dir = output_dir
        self.scale_file = scale_file
        self.dtype_spec = dtype_spec or {"some_column_name": str}

    def load_data(self):
        train_df = pd.read_csv(self.train_file, low_memory=False, dtype=self.dtype_spec)
        test_df = pd.read_csv(self.test_file, low_memory=False, dtype=self.dtype_spec)
        return train_df, test_df

    def fill_na(self, df):
        df = df.copy()
        df["type_y"] = df.get("type_y", pd.Series("Regular Day", index=df.index)).fillna("Regular Day")
        df["transactions"] = df.get("transactions", pd.Series(0, index=df.index)).fillna(0)
        df["dcoilwtico"] = df["dcoilwtico"].bfill()
        return df

    def add_dates(self, df):
        df = df.copy()
        df["date"] = pd.to_datetime(df["date"])
        df["year"] = df["date"].dt.year
        df["month"] = df["date"].dt.month
        df["day"] = df["date"].dt.day
        df["day_of_week"] = df["date"].dt.dayofweek
        df["is_weekend"] = (df["day_of_week"] >= 5).astype(int)
        df["day_of_year"] = df["date"].dt.dayofyear
        df["is_month_start"] = df["date"].dt.is_month_start.astype(int)
        df["is_month_end"] = df["date"].dt.is_month_end.astype(int)
        return df

    def add_interactions(self, df):
        df = df.copy()
        df["onpromotion_trend"] = df["onpromotion"] * df["day_of_year"]
        return df

    def encode_cyclical(self, df):
        df = df.copy()
        df["month_sin"] = np.sin(2 * np.pi * df["month"] / 12)
        df["month_cos"] = np.cos(2 * np.pi * df["month"] / 12)
        df["day_of_week_sin"] = np.sin(2 * np.pi * df["day_of_week"] / 7)
        df["day_of_week_cos"] = np.cos(2 * np.pi * df["day_of_week"] / 7)
        df.drop(["month", "day_of_week"], axis=1, inplace=True)
        return df

    def encode_and_scale(self, train_df, test_df):
        cat_columns = ["family", "state", "city", "type_x", "type_y"]

        combined = pd.concat([train_df, test_df], keys=["train", "test"])

        encoded = pd.get_dummies(combined[cat_columns], drop_first=True, dtype=int)
        combined = pd.concat([combined, encoded], axis=1)

        train_df = combined.xs("train").copy()
        test_df = combined.xs("test").copy()

        print(f"DataFrame cols: {train_df.columns.to_list()}")
        
        scale_columns = [
            col for col in train_df.columns
            if any(x in col for x in [
                "onpromotion_trend",
                "dcoilwtico", "transactions"
            ])
        ]
        scaler = MinMaxScaler()
        train_df[scale_columns] = scaler.fit_transform(train_df[scale_columns])
        test_df[scale_columns] = scaler.transform(test_df[scale_columns])

        os.makedirs(self.output_dir, exist_ok=True)
        joblib.dump(scaler, self.scale_file)

        return train_df, test_df

    def save(self, train_df, test_df):
        train_df.to_csv(Path(self.output_dir) / "train_final.csv", index=False)
        test_df.to_csv(Path(self.output_dir) / "test_final.csv", index=False)

    def run(self):
        train_df, test_df = self.load_data()
        train_df, test_df = self.fill_na(train_df), self.fill_na(test_df)

        train_df, test_df = self.add_dates(train_df), self.add_dates(test_df)

        train_df = self.add_interactions(train_df)
        test_df = self.add_interactions(test_df)

        train_df, test_df = self.encode_cyclical(train_df), self.encode_cyclical(test_df)

        train_df, test_df = self.encode_and_scale(train_df, test_df)

        train_df["is_holiday"] = train_df["type_y_Holiday"]

        self.save(train_df, test_df)

        print(f"Final shapes -> Train: {train_df.shape}, Test: {test_df.shape}")
        print("✅ Done: Final files saved!")


In [45]:
config_manager = ConfigurationManager(CONFIG_FILE_PATH)
feature_config = config_manager.get_feature_engineering_and_data_transformation_config()

fe = FeatureEngineeringAndDataTransformation(
    train_file=Path(feature_config.input_train_file),
    test_file=Path(feature_config.input_test_file),
    output_dir=feature_config.root_dir,
    scale_file=Path(feature_config.scaler_file),
)

fe.run()

[2025-12-01 20:48:00,548: INFO: main_utils: yaml file: config\config.yaml loaded successfully]
[2025-12-01 20:48:00,550: INFO: main_utils: created directory at: artifacts]
[2025-12-01 20:48:00,552: INFO: main_utils: created directory at: artifacts\features_dataTransformation]
Final shapes -> Train: (3013362, 102), Test: (28512, 101)
✅ Done: Final files saved!
